In [7]:
### Import libraries
import openai
import os
import time
from datetime import datetime
import pandas as pd
import clingo
from clingo.control import Control
import json
from yaml import load, Loader
from helper import *
from stories2 import *
from projects.creek import test

In [8]:
with open("config.yml","r") as config:
    data = load(config, Loader=Loader)
    api_key = data["apikey"]
    openai.api_key  = api_key

In [9]:
projects = [sudoku,seeknumbers,minotaur,creek]

In [10]:
for i in range(len(projects)):
    version = 52 # Fifth method of Variation
    copy_problem = dict(projects[i])
    example_1 = projects[i-1]
    example_2 = projects[i-2]
    story = copy_problem["story"]
    story_1 = example_1["story"]
    story_2 = example_2["story"]
    print(f'{story=}, {story_1=}, {story_2=}.')
    generation_rules = produce_generation_rules(example_1,example_2,copy_problem)
    definition_rules = produce_definition_rules(example_1,example_2,copy_problem)
    asp_code = [generation_rules,definition_rules]
    answer = '\n'.join(asp_code)
    name_of_file = os.path.join("generated_solutions",story+"_from_"+story_1+"_and_"+story_2+"_v"+str(version)+".lp")
    with open(name_of_file,"w") as file:
        file.write(answer)
    print("="*70)    

story='sudoku', story_1='creek', story_2='minotaur'.
% Define the possible digits
digit(1). digit(2). digit(3). digit(4). digit(5). digit(6). digit(7). digit(8). digit(9).

% Define the possible rows and columns
row(1). row(2). row(3). row(4). row(5). row(6). row(7). row(8). row(9).
col(1). col(2). col(3). col(4). col(5). col(6). col(7). col(8). col(9).

% Define the possible boxes
box(1). box(2). box(3). box(4). box(5). box(6). box(7). box(8). box(9).

% Define the possible values for each cell
1 { match(Ir,Ic,D) : digit(D) } 1 :- row(Ir), col(Ic).

% Define the constraint that the same number cannot appear twice in the same row
:- match(Ir,Ic,D), match(Ir,Ic2,D), col(Ic), col(Ic2), Ic != Ic2.

% Define the constraint that the same number cannot appear twice in the same column
:- match(Ir,Ic,D), match(Ir2,Ic,D), row(Ir), row(Ir2), Ir != Ir2.

% Define the constraint that the same number cannot appear twice in the same box
:- match(Ir,Ic,D), match(Ir2,Ic2,D), row(Ir), row(Ir2), col(Ic)

In [12]:
name_of_file = os.path.join("generated_solutions","creek_from_minotaur_and_seeknumbers_v5.lp")

In [25]:
def asp_try(asp_file,instance):
    solution_lines = []
    task = ""
    solution = ""
    models = None
    success = 0
    try:
        ctl = Control(arguments=["--opt-mode=opt"])
        ctl.load(asp_file)
        ctl.load(instance)
        ctl.add("base", [], solution)
        parts = [("base", [])]
        ctl.ground(parts)      
        try:
            with ctl.solve(yield_=True) as handle:
                models = list(iter(handle))
                if len(models) > 0:
                    success = 1
                    for m in models:
                        print(m.symbols(True,True,True))
        except:
            print("Unsatisfiable")
    except:
        print("Parsing or Safety Issues")
    return success, models

In [26]:
files = [x for x in os.listdir("generated_solutions") if "v5" in x]
print(files)

['creek_from_creek_and_minotaur_v5.lp', 'creek_from_minotaur_and_seeknumbers_v5.lp', 'minotaur_from_creek_and_minotaur_v5.lp', 'minotaur_from_seeknumbers_and_sudoku_v5.lp', 'seeknumbers_from_creek_and_minotaur_v5.lp', 'seeknumbers_from_sudoku_and_creek_v5.lp', 'sudoku_from_creek_and_minotaur_v5.lp']


In [27]:
instances_dict ={"sudoku":os.path.join("projects","sudoku","instances","ex01.lp"),
                 "seeknumbers":os.path.join("projects","seeknumbers","instances","ex01.lp"),
                 "minotaur":os.path.join("projects","minotaur","instances","level01.lp"),
                 "creek":os.path.join("projects","creek","instances","ex01.lp")
                }

In [28]:
for file in files:
    story = file.split("_")[0]
    instance = instances_dict[story]
    success, models = asp_try(name_of_file, instance)
    print(f"{success=}. {models=}")

Parsing or Safety Issues
success=0. models=None
Parsing or Safety Issues
success=0. models=None
Parsing or Safety Issues
success=0. models=None
Parsing or Safety Issues
success=0. models=None
Parsing or Safety Issues
success=0. models=None
Parsing or Safety Issues
success=0. models=None
Parsing or Safety Issues
success=0. models=None
